In [562]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-tp-17010420/train.csv
/kaggle/input/2022-ai-tp-17010420/test.csv
/kaggle/input/2022-ai-tp-17010420/sample_submit.csv


In [563]:
import numpy as np 
import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
import random
from sklearn.preprocessing import StandardScaler

In [564]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed=777
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if device == 'cuda':
    torch.cuda.manual_seed(777)
    torch.cuda.manual_seed_all(777)

torch.backends.cudnn.benchmark=False
torch.backends.cudnn.deterministic=True

In [565]:
train_data=pd.read_csv('/kaggle/input/2022-ai-tp-17010420/train.csv')
train_data

,도로 종류,가해자 차종,발생년도,횟수
0,시도,자전거,2011,626
1,시도,농기계,2008,71
2,시도,원동기장치자전거,2014,1871
3,일반국도,건설기계,2017,393
4,고속국도,건설기계,2012,21
...,...,...,...,...
625,군도,화물차,2014,1625
626,군도,승합차,2013,372
627,지방도,농기계,2006,95
628,시도,특수차,2020,324


In [566]:
train_data=pd.get_dummies(train_data,columns=['도로 종류','가해자 차종','발생년도'])
train_data

,횟수,도로 종류_고속국도,도로 종류_군도,도로 종류_시도,도로 종류_일반국도,도로 종류_지방도,도로 종류_특별광역시도,가해자 차종_건설기계,가해자 차종_농기계,가해자 차종_승용차,...,발생년도_2012,발생년도_2013,발생년도_2014,발생년도_2015,발생년도_2016,발생년도_2017,발생년도_2018,발생년도_2019,발생년도_2020,발생년도_2021
0,626,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,71,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1871,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,393,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,21,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,1625,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
626,372,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
627,95,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
628,324,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [567]:
train_data.iloc[:,0]

0       626
1        71
2      1871
3       393
4        21
       ... 
625    1625
626     372
627      95
628     324
629     468
Name: 횟수, Length: 630, dtype: int64

In [568]:
x_test=pd.read_csv('/kaggle/input/2022-ai-tp-17010420/test.csv')
x_test

,도로 종류,가해자 차종,발생년도
0,지방도,건설기계,2007
1,고속국도,승합차,2016
2,고속국도,자전거,2010
3,일반국도,농기계,2009
4,일반국도,이륜차,2019
...,...,...,...
266,일반국도,건설기계,2013
267,특별광역시도,건설기계,2011
268,지방도,자전거,2019
269,특별광역시도,이륜차,2011


In [569]:
x_test=pd.get_dummies(x_test,columns=['도로 종류','가해자 차종','발생년도'])
x_test

,도로 종류_고속국도,도로 종류_군도,도로 종류_시도,도로 종류_일반국도,도로 종류_지방도,도로 종류_특별광역시도,가해자 차종_건설기계,가해자 차종_농기계,가해자 차종_승용차,가해자 차종_승합차,...,발생년도_2012,발생년도_2013,발생년도_2014,발생년도_2015,발생년도_2016,발생년도_2017,발생년도_2018,발생년도_2019,발생년도_2020,발생년도_2021
0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,0,0,0,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
267,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
268,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
269,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [570]:
x_train=train_data.iloc[:,1:]
y_train=np.array(train_data.iloc[:,0]).reshape(630,1)
y_train=np.log1p(y_train)

In [571]:
x_train

,도로 종류_고속국도,도로 종류_군도,도로 종류_시도,도로 종류_일반국도,도로 종류_지방도,도로 종류_특별광역시도,가해자 차종_건설기계,가해자 차종_농기계,가해자 차종_승용차,가해자 차종_승합차,...,발생년도_2012,발생년도_2013,발생년도_2014,발생년도_2015,발생년도_2016,발생년도_2017,발생년도_2018,발생년도_2019,발생년도_2020,발생년도_2021
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
626,0,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
627,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
628,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [572]:
x_test

,도로 종류_고속국도,도로 종류_군도,도로 종류_시도,도로 종류_일반국도,도로 종류_지방도,도로 종류_특별광역시도,가해자 차종_건설기계,가해자 차종_농기계,가해자 차종_승용차,가해자 차종_승합차,...,발생년도_2012,발생년도_2013,발생년도_2014,발생년도_2015,발생년도_2016,발생년도_2017,발생년도_2018,발생년도_2019,발생년도_2020,발생년도_2021
0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,0,0,0,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
267,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
268,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
269,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [573]:
x_train = torch.FloatTensor(x_train.values).to(device)
y_train = torch.FloatTensor(y_train).to(device)
x_test = torch.FloatTensor(x_test.values).to(device)

In [574]:
learning_rate = 0.0001
training_epochs = 7900
node_num=512

In [575]:
class NN(torch.nn.Module):
    def __init__(self):
        super(NN, self).__init__() 

        self.linear1 = torch.nn.Linear(32, node_num, bias=True)
        self.linear2 = torch.nn.Linear(node_num, node_num, bias=True)
        self.linear3 = torch.nn.Linear(node_num, node_num, bias=True)
        self.linear4 = torch.nn.Linear(node_num, node_num, bias=True)
        self.linear5 = torch.nn.Linear(node_num, 1, bias=True)
        self.tanh=torch.nn.Tanh()
        self.relu = torch.nn.ReLU()

        torch.nn.init.xavier_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
        torch.nn.init.xavier_uniform_(self.linear3.weight)
        torch.nn.init.xavier_uniform_(self.linear4.weight)
        torch.nn.init.xavier_uniform_(self.linear5.weight)

    def forward(self,x):
        out = self.linear1(x)
        out = self.tanh(out)
        out = self.linear2(out)
        out = self.tanh(out)
        out = self.linear3(out)
        out = self.tanh(out)
        out = self.linear4(out)
        out = self.relu(out)
        out = self.linear5(out)
        return out


model = NN().to(device)

In [576]:
loss = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

In [577]:
#모델을 따로 생성했을 경우 학습 시 아래와 같이 명시해야 된다.
model.train()

for epoch in range(training_epochs):

    optimizer.zero_grad()

    hypothesis = model(x_train)
    cost = loss(hypothesis, y_train)

    cost.backward()
    optimizer.step()

    if epoch%100==0:
        print('Epoch: {:04d} // cost:{:.16f}'.format(epoch+1,cost))

print('Learning finished')

Epoch: 0001 // cost:44.0350379943847656
Epoch: 0101 // cost:0.8463783860206604
Epoch: 0201 // cost:0.6508518457412720
Epoch: 0301 // cost:0.2968400418758392
Epoch: 0401 // cost:0.1595828086137772
Epoch: 0501 // cost:0.1177073791623116
Epoch: 0601 // cost:0.1003989055752754
Epoch: 0701 // cost:0.0857568755745888
Epoch: 0801 // cost:0.0713522806763649
Epoch: 0901 // cost:0.0529657080769539
Epoch: 1001 // cost:0.0358500592410564
Epoch: 1101 // cost:0.0255608577281237
Epoch: 1201 // cost:0.0201273821294308
Epoch: 1301 // cost:0.0161758083850145
Epoch: 1401 // cost:0.0127888470888138
Epoch: 1501 // cost:0.0101640867069364
Epoch: 1601 // cost:0.0081992698833346
Epoch: 1701 // cost:0.0066353157162666
Epoch: 1801 // cost:0.0053687836043537
Epoch: 1901 // cost:0.0043846485204995
Epoch: 2001 // cost:0.0035922848619521
Epoch: 2101 // cost:0.0029239037539810
Epoch: 2201 // cost:0.0023604636080563
Epoch: 2301 // cost:0.0018987182993442
Epoch: 2401 // cost:0.0015182577772066
Epoch: 2501 // cost:0.00

In [578]:
with torch.no_grad():
    model.eval()

    prediction = model(x_test)

In [579]:
# csv 변환
prediction = prediction.cpu().numpy()
#prediction=sc_y.inverse_transform(prediction)
prediction=np.expm1(prediction)
submit = pd.read_csv("/kaggle/input/2022-ai-tp-17010420/sample_submit.csv")
submit['predict'] = prediction
submit.to_csv('/kaggle/working/submission.csv',index=False)